In [37]:
using LinearAlgebra, BenchmarkTools, SparseArrays, Plots

# Discrete state space

Construct state and action space

In [105]:
S = collect(0:2:1000);
N = size(S,1);
A = collect(0:2:1000); # Action space is how many units I can extract, let this be the same as the state space 

indexed_states = zeros(size(S,1), size(A,1));


Define utility functions

In [3]:
U1 = function(x)
   2 * x^0.5 
end

U2 = function(x)
    5 * x - 0.05 * x ^ 2
end

#3 (generic function with 1 method)

Flow utility

In [4]:
find_flow = function(ut)
    flow_U = zeros(size(S,1), size(A,1));
    for i in 1:size(S,1) #stock 
        for j in 1:size(A,1) #extraction
            if j <= i 
                if ut == 1 
                    flow_U[i,j] = U1(A[j]) # this is the flow utility at time t if I extract j with stock i remaining
                elseif ut == 2
                    flow_U[i,j] = U2(A[j])
                end
            else 
               flow_U[i,j] = -99999 # if j > i, i.e. extraction > stock, utility is -inf  
            end
        end
    end
    
    return flow_U
end


#5 (generic function with 1 method)

Find index of state in t+1 given action in t

In [5]:
state_index_t1 = zeros(size(S,1), size(A,1))

for i in 1:size(S,1)
    for j in 1:size(A,1)
        if j <= i
            left = S[i] - A[j]
            state_index_t1[i,j] = findall(x->x==left, S)[1]
        else
            state_index_t1[i,j] = 1 # it will be stock = 0 if I try to extract more than I have
         end
    end
end


Now construct a transition matrix

In [6]:
# you need to find the state + action index of each column index
colindex = []
iter = 0 
for j in 1:size(A, 1)
    for i in 1:size(S,1)
        iter += 1 
        push!(colindex, [iter, i, j])
    end
end

# i,j = 1 iff state_index_t1 says I transition to j given action A 
# (this is a horrible way to constructing my sparse matrix but whatever)
v = Float64[]
r = []
c = []
for i in 1:size(S,1)
    for j in 1:size(colindex,1)
        if state_index_t1[i,colindex[j][3]] == colindex[j][2]
            push!(r, i)
            push!(c, colindex[j][1])
            push!(v, 1)
        end
    end
end

In [7]:
T = sparse(r, c, v)

501×250501 SparseMatrixCSC{Float64, Int64} with 251001 stored entries:
⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛

Now let's try iterating

In [41]:
iterate_wrapper = function(utility_fn, discount, tol)
    U = find_flow(utility_fn)
    δ = discount
    err = 1
    v = zeros(size(S,1),1)
    
    iter = 0
    while err > tol
        iter += 1
        v_old = v

        # calculate v_next given each action
        Vₙₑₓₜ = zeros(size(S,1), size(A,1))
        for x in 1:size(A,1)-1
            Vₙₑₓₜ[:,x] = T[:, 1+size(S,1) * (x-1):501+size(S,1) * (x-1)] * U[:,x]
        end

        #println(Vₙₑₓₜ)

        # find optimal action over each state and action 
        v = maximum(U + δ .* Vₙₑₓₜ, dims=2) 
        gₖ = argmax(U + δ .* Vₙₑₓₜ, dims=2)

        # something is definitely wrong here we are not using the new optimal value function
        #U = 

        #println(maximum(abs.(v .- v_old)))
        err = maximum(abs.(v .- v_old))
    end
    
    return [v, gₖ, err, iter]
end

Find optimal transition path

In [ ]:
simulate_paths = function(oc, time)
    stock = 1000
    df = []
    
    for t=1:time
        state_i = findall(x->x==stock, S)
        action_j = oc[state_i, 2][1]
        stock = stock - A[action_j]
        push!(df, [t, stock, A[action_j]])
    end
    
    # how to calculate prices?
    
    return df
end

# Interpolating between states

In [113]:
S = collect(0:2:1000);
N = size(S,1);
A = collect(range(0, sqrt(1000), 501)).^2;


Find state transition matrix

In [157]:
# this feels so inefficient...
state_index_t1 = Array{Array}(undef, size(S,1), size(A,1))

for i in 1:size(S,1)
    for j in 1:size(A,1)
        if j <= i
            left = S[i] - A[j]
            # if action lines up exactly with a state 
            if length(findall(x->x==left, S)) != 0 
                state_index_t1[i,j] = [findall(x->x==left, S)[1]]
            else 
                
                # try to extract the closest state space on the left and right
                difference = S .- left 
                posmin = Inf;
                negmax = -Inf;
                for d in 1:length(difference)
                    if 0 <= difference[d] <= posmin
                        posmin=difference[d] 
                    end
                    
                    if 0 >= difference[d] >= negmax
                        negmax=difference[d]
                    end
                end
                pos = findall(x->x==posmin, difference)[1]
                neg = findall(x->x==negmax, difference)[1]
                
                # get the weights since we already calculated the difference
                sumdiff = abs(difference[pos]) + abs(difference[neg])
                negweight = abs(difference[neg])/sumdiff 
                posweight = abs(difference[pos])/sumdiff
                
                state_index_t1[i,j] = [neg, pos, negweight, posweight]
            end 
        else
            state_index_t1[i,j] = [1] # it will be stock = 0 if I try to extract more than I have
        end
    end
end


Interpolating the transition matrix

In [114]:
colindex = []
iter = 0 
for j in 1:size(A, 1)
    for i in 1:size(S,1)
        iter += 1 
        push!(colindex, [iter, i, j])
    end
end

In [115]:
v = Float64[]
r = []
c = []
for i in 1:size(S,1)
    for j in 1:size(colindex,1)
        
        if state_index_t1[i,colindex[j][3]] == colindex[j][2]
            push!(r, i)
            push!(c, colindex[j][1])
            push!(v, 1)
        end
    end
end

251001-element Vector{Any}:
 [1, 1, 1]
 [2, 2, 1]
 [3, 3, 1]
 [4, 4, 1]
 [5, 5, 1]
 [6, 6, 1]
 [7, 7, 1]
 [8, 8, 1]
 [9, 9, 1]
 [10, 10, 1]
 [11, 11, 1]
 [12, 12, 1]
 [13, 13, 1]
 ⋮
 [250990, 490, 501]
 [250991, 491, 501]
 [250992, 492, 501]
 [250993, 493, 501]
 [250994, 494, 501]
 [250995, 495, 501]
 [250996, 496, 501]
 [250997, 497, 501]
 [250998, 498, 501]
 [250999, 499, 501]
 [251000, 500, 501]
 [251001, 501, 501]

In [ ]:
# you need to find the state + action index of each column index
colindex = []
iter = 0 
for j in 1:size(A, 1)
    for i in 1:size(S,1)
        iter += 1 
        push!(colindex, [iter, i, j])
    end
end

# i,j = 1 iff state_index_t1 says I transition to j given action A 
# (this is a horrible way to constructing my sparse matrix but whatever)
v = Float64[]
r = []
c = []
for i in 1:size(S,1)
    for j in 1:size(colindex,1)
        if state_index_t1[i,colindex[j][3]] == colindex[j][2]
            push!(r, i)
            push!(c, colindex[j][1])
            push!(v, 1)
        end
    end
end